In [8]:
import pandas as pd
from pandas import DataFrame
from sqlalchemy import create_engine
import sqlalchemy

We load the weather data below. The fields for this dataset are enumerated [here](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/ghcn-daily-by_year-format.rtf).

In [9]:
weather_data = pd.read_csv('data/1900.csv',
                           header=1,
                           index_col=False,
                           names=['station_identifier',
                                  'measurement_date',
                                  'measurement_type',
                                  'measurement_flag',
                                  'quality_flag',
                                  'source_flag',
                                  'observation_time'],
                           parse_dates=['measurement_date'])


In [10]:
weather_data.head()

,station_identifier,measurement_date,measurement_type,measurement_flag,quality_flag,source_flag,observation_time
0,AGE00135039,1900-01-01,TMIN,140,NaN,NaN,E
1,AGE00135039,1900-01-01,PRCP,0,NaN,NaN,E
2,AGE00147705,1900-01-01,TMAX,254,NaN,NaN,E
3,AGE00147705,1900-01-01,TMIN,153,NaN,NaN,E
4,AGE00147705,1900-01-01,PRCP,2,NaN,NaN,E


There are a [large number](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) of categories of weather data in the dataset. For simplicity, we only want to load the data from the 5 "core" weather categories into the database for further analysis:

* PRCP : Precipitation (tenths of mm)
* SNOW : Snowfall (mm)
* SNWD : Snow depth (mm)
* TMAX : Maximum temperature (tenths of degrees C)
* TMIN : Minimum temperature (tenths of degrees C)

We also want to cull a few columns from the DataFrame.

In [11]:
weather_data_subset = weather_data[weather_data.measurement_type.isin(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN'])][['station_identifier', 'measurement_date', 'measurement_type', 'measurement_flag']]

In [12]:
weather_data_subset.head()

,station_identifier,measurement_date,measurement_type,measurement_flag
0,AGE00135039,1900-01-01,TMIN,140
1,AGE00135039,1900-01-01,PRCP,0
2,AGE00147705,1900-01-01,TMAX,254
3,AGE00147705,1900-01-01,TMIN,153
4,AGE00147705,1900-01-01,PRCP,2


This cuts down the total count of records by about 30%.

Now, let's write the weather data to our DB. If you setup the Postgres DB as noted in the README, this should instantiate a connection to the database with your local unix username.

If you've configured another Postgres user with a username / password, please fill in the appropriate credentials using the [SQL Alchemy connection string](http://docs.sqlalchemy.org/en/latest/core/engines.html).

In [17]:
dbschema = 'ghcnd'
user = "root"
password = "root"
host = "localhost"
port = 5432
db = "ghcnd"
cmd = f'postgresql://{user}:{password}@{host}:{port}/{db}'
engine = create_engine(cmd, connect_args={'options': f'-csearch_path={dbschema}'})
engine.connect()

table_name = 'weather_data'
# The to_sql method defaults to bigint for integer types here, which are larger than we need. 
# This manually sets the datatypes of the columns we need to override
column_type_dict = {'measurement_flag': sqlalchemy.types.Integer}
# Writing all the data to the DB at once will cause this notebook to crash.
# We pass a large integer to the chunksize parameter to chunk the writing of records
weather_data_subset.to_sql(table_name, engine,schema="public", chunksize=100000, index_label='id', dtype=column_type_dict)

45571

Now, let's process and read in the metadata - which contains the (lat, long) - tied to each weather station.

In [18]:
station_metadata = pd.read_csv('data/ghcnd-stations.txt', 
                           sep='\s+',  # Fields are separated by one or more spaces
                           usecols=[0, 1, 2, 3],  # Grab only the first 4 columns
                           na_values=[-999.9],  # Missing elevation is noted as -999.9
                           header=None,
                           names=['station_id', 'latitude', 'longitude', 'elevation'])

In [19]:
station_metadata.head()

,station_id,latitude,longitude,elevation
0,ACW00011604,17.1167,-61.7833,10.1
1,ACW00011647,17.1333,-61.7833,19.2
2,AE000041196,25.3330,55.5170,34.0
3,AEM00041194,25.2550,55.3640,10.4
4,AEM00041217,24.4330,54.6510,26.8


How many stations do we have with missing elevation?

In [20]:
len(station_metadata[station_metadata['elevation'].isnull()])

4621

Write the metadata to the DB:

In [22]:
metadata_table_name = 'station_metadata'
station_metadata.to_sql(metadata_table_name, engine, schema="public", index_label='id')

346

Finally, we want to label the 5 types of weather data noted above with human-readable names. We create a dictionary mapping each label to its description, create a Data Frame from that, and write it to the database:

In [23]:
weather_type_dict = {'PRCP': 'Precipitation', 'SNOW': 'Snowfall', 'SNWD': 'Snow Depth', 
                     'TMAX': 'Maximum temperature', 'TMIN': 'Minimum temperature'}
weather_type_df = DataFrame(weather_type_dict.items(), columns=['weather_type', 'weather_description'])
description_table_name = 'weather_types'
weather_type_df.to_sql(description_table_name, engine, schema="public", index_label='id')

5